In [8]:
import mxnet as mx

In [13]:
import numpy as np

In [1]:
from mxnet.gluon.model_zoo import vision

In [2]:
import json
from collections import OrderedDict

In [3]:
def lord_labels():
    """
    """
    filename = 'imagenet_class_index.json'
    with open(filename, 'r') as fr:
        labels = json.load(fr, object_pairs_hook=OrderedDict)
    return [v[-1] for k, v in labels.items()]

In [4]:
labels = lord_labels()

In [101]:
img_filename = 'C:/Users/emile/Pictures/mug227.png'
# img_filename = 'C:/Users/emile/Pictures/test_0.png'

In [93]:
model = vision.resnet152_v2(pretrained=True)

In [94]:
def transform(image):
    resized = mx.image.resize_short(image, 224) #minimum 224x224 images
    cropped, crop_info = mx.image.center_crop(resized, (224, 224))
    normalized = mx.image.color_normalize(cropped.astype(np.float32)/255,
                                      mean=mx.nd.array([0.485, 0.456, 0.406]),
                                      std=mx.nd.array([0.229, 0.224, 0.225])) 
    # the network expect batches of the form (N,3,224,224)
    transposed = normalized.transpose((2,0,1))  # Transposing from (224, 224, 3) to (3, 224, 224)
    batchified = transposed.expand_dims(axis=0) # change the shape from (3, 224, 224) to (1, 3, 224, 224)
    return batchified

In [102]:
image = mx.image.imread(img_filename)

In [103]:
data = transform(image)

In [104]:
prediction = model(data).softmax()[0]

In [105]:
top_5 = prediction.topk(k=5).asnumpy().astype(dtype=np.int32)

In [106]:
top_5

array([967, 504, 968, 550, 505])

In [107]:
for i in top_5:
    label = labels[i]
    prob = prediction[i].asnumpy().item()
    print(f'label: {label}, prob: {prob}')

label: espresso, prob: 0.4120803773403168
label: coffee_mug, prob: 0.3189449608325958
label: cup, prob: 0.12735049426555634
label: espresso_maker, prob: 0.04224308952689171
label: coffeepot, prob: 0.03870426118373871
